In [1]:
! pip install flax tensorflow tensorflow_datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 17.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 139.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 51.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 51.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 53

In [3]:
! pip install "jax[tpu]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 MB 29.7 MB/s eta 0:00:0000:0100:01


In [3]:
import jax
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=2, process_index=0, coords=(2,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(3,0,0), core_on_chip=0),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=6, process_index=0, coords=(2,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(3,1,0), core_on_chip=0)]

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import jax
import jax.numpy as jnp
import flax.linen as nn
from flax.training import train_state

import functools

import flax.linen.attention as attention

import numpy as np

import optax

import time



2025-04-12 01:54:40.969001: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-12 01:54:40.972096: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-12 01:54:40.982209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744422880.999972   25942 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744422881.005215   25942 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744422881.017742   25942 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
BATCH_IN_SEQUENCES = 384
SEQUENCE_LENGTH = 128

VOCAB_DIM = 256
EMBED_DIM = 512
FF_DIM = 2048

NUM_HEADS = 4
HEAD_DIM = 128

LAYERS = 2

HEAD_DEPTH = 128
NUM_HEADS = 4

LEARNING_RATE = 1e-3

FSDP = 4 # 8 OR 4
TENSOR = 1 # 1 OR 2


def attention_ourselves(_Q, _K, _V):
    _weights_unnormalized = jax.numpy.einsum("BSHD,BTHD->BHST", _Q, _K)
    _weights_unnormalized_to_zero_out = jax.numpy.triu( jax.numpy.ones((SEQUENCE_LENGTH,SEQUENCE_LENGTH), jax.numpy.bfloat16), 1)
    _weights = jax.nn.softmax(_weights_unnormalized - 1e6 * _weights_unnormalized_to_zero_out)  ### Creating something of size (B,HEADS, SEQUENCE, SEQUENCE)
    #print(f"{_weights.size=}")
    output = jax.numpy.einsum("BHST,BTHD->BSHD", _weights, _V)

    return output

class OurModel(nn.Module):
  @nn.compact
  def __call__(self, x):
    '''
        x is [BATCH, SEQUENCE]
    '''
    embedding = self.param(
        'embedding',
        nn.with_partitioning(nn.initializers.normal(1), ("tp", "fsdp")),
        (VOCAB_DIM, EMBED_DIM),
        jnp.float32,
    )
    x = embedding[x] ##OUTPUT should be [BATCH, SEQUENCE, EMBED]

    positional_embedding = self.param(
        'positional_embedding',
        nn.with_partitioning(nn.initializers.normal(1), (None, None, "fsdp")),
        (1, SEQUENCE_LENGTH, EMBED_DIM),
        jnp.float32,
    )

    x += positional_embedding


    for i in range(LAYERS):
      feedforward = self.param(
          'feedforward_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, FF_DIM),
          jnp.float32,
      )
      x = x @ feedforward
      x = jax.nn.relu(x)
      embed = self.param(
          'embed_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('tp', 'fsdp')),
          (FF_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = x @ embed
      x = jax.nn.relu(x)

      q_proj = self.param(
          'qproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      q = jnp.einsum("BSE,EHD->BSHD",x, q_proj )

      k_proj = self.param(
          'kproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      k = jnp.einsum("BSE,EHD->BSHD",x, k_proj )

      v_proj = self.param(
          'vproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      v = jnp.einsum("BSE,EHD->BSHD",x, v_proj )

      o = attention_ourselves(q,k,v)

      o_proj = self.param(
          'oproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (NUM_HEADS, HEAD_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = jnp.einsum("BSHD,HDE->BSE",o, o_proj )

    return x @ embedding.T

def convert_to_ascii(string_array, max_length):
  result = np.zeros((len(string_array), max_length), dtype=np.uint8)
  for i, string in enumerate(string_array):
    for j, char in enumerate(string):
      if j >= SEQUENCE_LENGTH:
         break
      result[i, j] = char
  return result

def input_to_output(np_array):
   zero_array = np.zeros( (BATCH_IN_SEQUENCES,SEQUENCE_LENGTH), dtype = jnp.uint8)
   zero_array[:, 1:SEQUENCE_LENGTH] = np_array[:, 0:SEQUENCE_LENGTH-1]
   return zero_array

def calculate_loss(params, model, inputs, outputs):
   proposed_outputs = model.apply(params, inputs)
   one_hot = jax.nn.one_hot(outputs, VOCAB_DIM)
   loss = optax.softmax_cross_entropy(proposed_outputs, one_hot)
   return jnp.mean(loss)


def step(state, model, inputs, outputs):
   loss, grad = jax.value_and_grad(calculate_loss)(state.params, model, inputs, outputs)
   state = state.apply_gradients(grads = grad)
   return loss, state


mesh = jax.sharding.Mesh(np.reshape(  jax.devices(), (FSDP,TENSOR)), ["fsdp", "tp"])

ds = tfds.load('lm1b', split='train', shuffle_files=False)
ds = ds.batch(BATCH_IN_SEQUENCES)

rngkey = jax.random.key(0)
model = OurModel()

shaped_init = jax.eval_shape( functools.partial(model.init, rngkey), jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))
state_sharding = nn.get_sharding(shaped_init, mesh)
_params = jax.jit(model.init, out_shardings = state_sharding)(rngkey, jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))

tx = optax.adam(learning_rate = LEARNING_RATE)
state = train_state.TrainState.create(
    apply_fn = model.apply,
    params = _params,
    tx = tx
)

iter = 0
static_step = jax.jit(step, static_argnums=1)

last_step_time = time.time()
stepnum = 0

for example in ds:
    outputs = convert_to_ascii(example['text'].numpy(), SEQUENCE_LENGTH)
    inputs = input_to_output(outputs)

    loss, state = static_step(state, model, inputs, outputs)
    #loss, state = jax.jit(step, static_argnums=1)(state, model, inputs, outputs)
    #loss, state = jax.jit(lambda x,y,z,a : step(x,y,z,a), static_argnums=1)(state, model, inputs, outputs)

    stepnum += 1

    if stepnum % 10 == 0:
      new_time = time.time()
      time_elapsed_seconds = (new_time-last_step_time)
      last_step_time = new_time
      print(f"{iter} -> {loss} {time_elapsed_seconds}")


    iter += 1


ValueError: cannot reshape array of size 8 into shape (4,1)

In [5]:
BATCH_IN_SEQUENCES = 384
SEQUENCE_LENGTH = 128

VOCAB_DIM = 256
EMBED_DIM = 512
FF_DIM = 2048

NUM_HEADS = 4
HEAD_DIM = 128

LAYERS = 2

HEAD_DEPTH = 128

LEARNING_RATE = 1e-3

FSDP = 8
TENSOR = 1


def attention_ourselves(_Q, _K, _V):
    _weights_unnormalized = jax.numpy.einsum("BSHD,BTHD->BHST", _Q, _K)
    _weights_unnormalized_to_zero_out = jax.numpy.triu( jax.numpy.ones((SEQUENCE_LENGTH,SEQUENCE_LENGTH), jax.numpy.bfloat16), 1)
    _weights = jax.nn.softmax(_weights_unnormalized - 1e6 * _weights_unnormalized_to_zero_out)  ### Creating something of size (B,HEADS, SEQUENCE, SEQUENCE)
    #print(f"{_weights.size=}")
    output = jax.numpy.einsum("BHST,BTHD->BSHD", _weights, _V)

    return output

class OurModel(nn.Module):
  @nn.compact
  def __call__(self, x):
    '''
        x is [BATCH, SEQUENCE]
    '''
    embedding = self.param(
        'embedding',
        nn.with_partitioning(nn.initializers.normal(1), ("tp", "fsdp")),
        (VOCAB_DIM, EMBED_DIM),
        jnp.float32,
    )
    x = embedding[x] ##OUTPUT should be [BATCH, SEQUENCE, EMBED]

    positional_embedding = self.param(
        'positional_embedding',
        nn.with_partitioning(nn.initializers.normal(1), (None, None, "fsdp")),
        (1, SEQUENCE_LENGTH, EMBED_DIM),
        jnp.float32,
    )

    x += positional_embedding


    for i in range(LAYERS):
      feedforward = self.param(
          'feedforward_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, FF_DIM),
          jnp.float32,
      )
      x = x @ feedforward
      x = jax.nn.relu(x)
      embed = self.param(
          'embed_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('tp', 'fsdp')),
          (FF_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = x @ embed
      x = jax.nn.relu(x)

      q_proj = self.param(
          'qproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      q = jnp.einsum("BSE,EHD->BSHD",x, q_proj )

      k_proj = self.param(
          'kproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      k = jnp.einsum("BSE,EHD->BSHD",x, k_proj )

      v_proj = self.param(
          'vproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      v = jnp.einsum("BSE,EHD->BSHD",x, v_proj )

      o = attention_ourselves(q,k,v)

      o_proj = self.param(
          'oproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (NUM_HEADS, HEAD_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = jnp.einsum("BSHD,HDE->BSE",o, o_proj )

    return x @ embedding.T # missing the softmax with probabilities for each token?

def convert_to_ascii(string_array, max_length):
  result = np.zeros((len(string_array), max_length), dtype=np.uint8)
  for i, string in enumerate(string_array):
    for j, char in enumerate(string):
      if j >= SEQUENCE_LENGTH:
         break
      result[i, j] = char
  return result

def input_to_output(np_array):
   zero_array = np.zeros( (BATCH_IN_SEQUENCES,SEQUENCE_LENGTH), dtype = jnp.uint8)
   zero_array[:, 1:SEQUENCE_LENGTH] = np_array[:, 0:SEQUENCE_LENGTH-1]
   return zero_array

def calculate_loss(params, model, inputs, outputs):
   proposed_outputs = model.apply(params, inputs)
   one_hot = jax.nn.one_hot(outputs, VOCAB_DIM)
   loss = optax.softmax_cross_entropy(proposed_outputs, one_hot)
   return jnp.mean(loss)


def step(state, model, inputs, outputs):
   loss, grad = jax.value_and_grad(calculate_loss)(state.params, model, inputs, outputs)
   state = state.apply_gradients(grads = grad)
   return loss, state


mesh = jax.sharding.Mesh(np.reshape(  jax.devices(), (FSDP,TENSOR)), ["fsdp", "tp"])

ds = tfds.load('lm1b', split='train', shuffle_files=False)
ds = ds.batch(BATCH_IN_SEQUENCES)

rngkey = jax.random.key(0)
model = OurModel()

shaped_init = jax.eval_shape( functools.partial(model.init, rngkey), jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))
state_sharding = nn.get_sharding(shaped_init, mesh)
_params = jax.jit(model.init, out_shardings = state_sharding)(rngkey, jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))

tx = optax.adam(learning_rate = LEARNING_RATE)
state = train_state.TrainState.create(
    apply_fn = model.apply,
    params = _params,
    tx = tx
)

iter = 0
static_step = jax.jit(step, static_argnums=1)

last_step_time = time.time()
stepnum = 0

for example in ds:
    outputs = convert_to_ascii(example['text'].numpy(), SEQUENCE_LENGTH)
    inputs = input_to_output(outputs)

    loss, state = static_step(state, model, inputs, outputs)
    # assert model_without_pos_embeddings("Sam and Erin") == model_without_pos_embeddings("Erin and Sam")
    # assert model_with_pos_embeddings("Sam and Erin") != model_with_pos_embeddings("Erin and Sam")
    #loss, state = jax.jit(step, static_argnums=1)(state, model, inputs, outputs)
    #loss, state = jax.jit(lambda x,y,z,a : step(x,y,z,a), static_argnums=1)(state, model, inputs, outputs)

    stepnum += 1

    if stepnum % 10 == 0:
      new_time = time.time()
      time_elapsed_seconds = (new_time-last_step_time)
      last_step_time = new_time
      print(f"{iter} -> {loss} {time_elapsed_seconds}")


    iter += 1


ValueError: One of pjit outputs with pytree key path result['params']['oproj_0'].value was given the sharding of NamedSharding(mesh=Mesh('fsdp': 8, 'tp': 1, axis_types=(Auto, Auto)), spec=PartitionSpec('fsdp', 'tp'), memory_kind=device), which implies that the global size of its dimension 0 should be divisible by 8, but it is equal to 4 (full shape: (4, 128, 512))

In [6]:
BATCH_IN_SEQUENCES = 384
SEQUENCE_LENGTH = 128

VOCAB_DIM = 256
EMBED_DIM = 512
FF_DIM = 2048

NUM_HEADS = 8
HEAD_DIM = 128

LAYERS = 2

HEAD_DEPTH = 128

LEARNING_RATE = 1e-3

FSDP = 8
TENSOR = 1


def attention_ourselves(_Q, _K, _V):
    _weights_unnormalized = jax.numpy.einsum("BSHD,BTHD->BHST", _Q, _K)
    _weights_unnormalized_to_zero_out = jax.numpy.triu( jax.numpy.ones((SEQUENCE_LENGTH,SEQUENCE_LENGTH), jax.numpy.bfloat16), 1)
    _weights = jax.nn.softmax(_weights_unnormalized - 1e6 * _weights_unnormalized_to_zero_out)  ### Creating something of size (B,HEADS, SEQUENCE, SEQUENCE)
    #print(f"{_weights.size=}")
    output = jax.numpy.einsum("BHST,BTHD->BSHD", _weights, _V)

    return output

class OurModel(nn.Module):
  @nn.compact
  def __call__(self, x):
    '''
        x is [BATCH, SEQUENCE]
    '''
    embedding = self.param(
        'embedding',
        nn.with_partitioning(nn.initializers.normal(1), ("tp", "fsdp")),
        (VOCAB_DIM, EMBED_DIM),
        jnp.float32,
    )
    x = embedding[x] ##OUTPUT should be [BATCH, SEQUENCE, EMBED]

    positional_embedding = self.param(
        'positional_embedding',
        nn.with_partitioning(nn.initializers.normal(1), (None, None, "fsdp")),
        (1, SEQUENCE_LENGTH, EMBED_DIM),
        jnp.float32,
    )

    x += positional_embedding


    for i in range(LAYERS):
      feedforward = self.param(
          'feedforward_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, FF_DIM),
          jnp.float32,
      )
      x = x @ feedforward
      x = jax.nn.relu(x)
      embed = self.param(
          'embed_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('tp', 'fsdp')),
          (FF_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = x @ embed
      x = jax.nn.relu(x)

      q_proj = self.param(
          'qproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      q = jnp.einsum("BSE,EHD->BSHD",x, q_proj )

      k_proj = self.param(
          'kproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      k = jnp.einsum("BSE,EHD->BSHD",x, k_proj )

      v_proj = self.param(
          'vproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      v = jnp.einsum("BSE,EHD->BSHD",x, v_proj )

      o = attention_ourselves(q,k,v)

      o_proj = self.param(
          'oproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (NUM_HEADS, HEAD_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = jnp.einsum("BSHD,HDE->BSE",o, o_proj )

    return x @ embedding.T # missing the softmax with probabilities for each token?

def convert_to_ascii(string_array, max_length):
  result = np.zeros((len(string_array), max_length), dtype=np.uint8)
  for i, string in enumerate(string_array):
    for j, char in enumerate(string):
      if j >= SEQUENCE_LENGTH:
         break
      result[i, j] = char
  return result

def input_to_output(np_array):
   zero_array = np.zeros( (BATCH_IN_SEQUENCES,SEQUENCE_LENGTH), dtype = jnp.uint8)
   zero_array[:, 1:SEQUENCE_LENGTH] = np_array[:, 0:SEQUENCE_LENGTH-1]
   return zero_array

def calculate_loss(params, model, inputs, outputs):
   proposed_outputs = model.apply(params, inputs)
   one_hot = jax.nn.one_hot(outputs, VOCAB_DIM)
   loss = optax.softmax_cross_entropy(proposed_outputs, one_hot)
   return jnp.mean(loss)


def step(state, model, inputs, outputs):
   loss, grad = jax.value_and_grad(calculate_loss)(state.params, model, inputs, outputs)
   state = state.apply_gradients(grads = grad)
   return loss, state


mesh = jax.sharding.Mesh(np.reshape(  jax.devices(), (FSDP,TENSOR)), ["fsdp", "tp"])

ds = tfds.load('lm1b', split='train', shuffle_files=False)
ds = ds.batch(BATCH_IN_SEQUENCES)

rngkey = jax.random.key(0)
model = OurModel()

shaped_init = jax.eval_shape( functools.partial(model.init, rngkey), jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))
state_sharding = nn.get_sharding(shaped_init, mesh)
_params = jax.jit(model.init, out_shardings = state_sharding)(rngkey, jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))

tx = optax.adam(learning_rate = LEARNING_RATE)
state = train_state.TrainState.create(
    apply_fn = model.apply,
    params = _params,
    tx = tx
)

iter = 0
static_step = jax.jit(step, static_argnums=1)

last_step_time = time.time()
stepnum = 0

for example in ds:
    outputs = convert_to_ascii(example['text'].numpy(), SEQUENCE_LENGTH)
    inputs = input_to_output(outputs)

    loss, state = static_step(state, model, inputs, outputs)
    # assert model_without_pos_embeddings("Sam and Erin") == model_without_pos_embeddings("Erin and Sam")
    # assert model_with_pos_embeddings("Sam and Erin") != model_with_pos_embeddings("Erin and Sam")
    #loss, state = jax.jit(step, static_argnums=1)(state, model, inputs, outputs)
    #loss, state = jax.jit(lambda x,y,z,a : step(x,y,z,a), static_argnums=1)(state, model, inputs, outputs)

    stepnum += 1

    if stepnum % 10 == 0:
      new_time = time.time()
      time_elapsed_seconds = (new_time-last_step_time)
      last_step_time = new_time
      print(f"{iter} -> {loss} {time_elapsed_seconds}")


    iter += 1
    if stepnum == 1000:
        break

2025-04-12 03:27:20.008887: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


9 -> 4.784549713134766 8.760738372802734
19 -> 3.8527989387512207 0.6371138095855713
29 -> 3.095243215560913 0.6934900283813477
39 -> 2.7191176414489746 0.6936655044555664
49 -> 2.645170211791992 0.6936140060424805
59 -> 2.5840907096862793 0.6946406364440918
69 -> 2.620162010192871 0.6965792179107666
79 -> 2.612311601638794 0.6946649551391602
89 -> 2.6059045791625977 0.6969215869903564
99 -> 2.552733898162842 0.6944339275360107
109 -> 2.5300450325012207 0.6907339096069336
119 -> 2.514218807220459 0.6982433795928955
129 -> 2.5057194232940674 0.6927556991577148
139 -> 2.566188097000122 0.6936328411102295
149 -> 2.501035213470459 0.6959996223449707
159 -> 2.4965708255767822 0.6950409412384033
169 -> 2.5686490535736084 0.6938967704772949
179 -> 2.6139049530029297 0.6937763690948486
189 -> 2.6034724712371826 0.6953089237213135
199 -> 2.512711524963379 0.6950550079345703
209 -> 2.5235447883605957 0.6927204132080078
219 -> 2.5821216106414795 0.6956758499145508
229 -> 2.5074775218963623 0.6943

In [7]:
BATCH_IN_SEQUENCES = 384
SEQUENCE_LENGTH = 128

VOCAB_DIM = 256
EMBED_DIM = 512
FF_DIM = 2048

NUM_HEADS = 4
HEAD_DIM = 128

LAYERS = 2

HEAD_DEPTH = 128

LEARNING_RATE = 1e-3

FSDP = 4
TENSOR = 2


def attention_ourselves(_Q, _K, _V):
    _weights_unnormalized = jax.numpy.einsum("BSHD,BTHD->BHST", _Q, _K)
    _weights_unnormalized_to_zero_out = jax.numpy.triu( jax.numpy.ones((SEQUENCE_LENGTH,SEQUENCE_LENGTH), jax.numpy.bfloat16), 1)
    _weights = jax.nn.softmax(_weights_unnormalized - 1e6 * _weights_unnormalized_to_zero_out)  ### Creating something of size (B,HEADS, SEQUENCE, SEQUENCE)
    #print(f"{_weights.size=}")
    output = jax.numpy.einsum("BHST,BTHD->BSHD", _weights, _V)

    return output

class OurModel(nn.Module):
  @nn.compact
  def __call__(self, x):
    '''
        x is [BATCH, SEQUENCE]
    '''
    embedding = self.param(
        'embedding',
        nn.with_partitioning(nn.initializers.normal(1), ("tp", "fsdp")),
        (VOCAB_DIM, EMBED_DIM),
        jnp.float32,
    )
    x = embedding[x] ##OUTPUT should be [BATCH, SEQUENCE, EMBED]

    positional_embedding = self.param(
        'positional_embedding',
        nn.with_partitioning(nn.initializers.normal(1), (None, None, "fsdp")),
        (1, SEQUENCE_LENGTH, EMBED_DIM),
        jnp.float32,
    )

    x += positional_embedding


    for i in range(LAYERS):
      feedforward = self.param(
          'feedforward_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, FF_DIM),
          jnp.float32,
      )
      x = x @ feedforward
      x = jax.nn.relu(x)
      embed = self.param(
          'embed_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('tp', 'fsdp')),
          (FF_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = x @ embed
      x = jax.nn.relu(x)

      q_proj = self.param(
          'qproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      q = jnp.einsum("BSE,EHD->BSHD",x, q_proj )

      k_proj = self.param(
          'kproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      k = jnp.einsum("BSE,EHD->BSHD",x, k_proj )

      v_proj = self.param(
          'vproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      v = jnp.einsum("BSE,EHD->BSHD",x, v_proj )

      o = attention_ourselves(q,k,v)

      o_proj = self.param(
          'oproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (NUM_HEADS, HEAD_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = jnp.einsum("BSHD,HDE->BSE",o, o_proj )

    return x @ embedding.T # missing the softmax with probabilities for each token?

def convert_to_ascii(string_array, max_length):
  result = np.zeros((len(string_array), max_length), dtype=np.uint8)
  for i, string in enumerate(string_array):
    for j, char in enumerate(string):
      if j >= SEQUENCE_LENGTH:
         break
      result[i, j] = char
  return result

def input_to_output(np_array):
   zero_array = np.zeros( (BATCH_IN_SEQUENCES,SEQUENCE_LENGTH), dtype = jnp.uint8)
   zero_array[:, 1:SEQUENCE_LENGTH] = np_array[:, 0:SEQUENCE_LENGTH-1]
   return zero_array

def calculate_loss(params, model, inputs, outputs):
   proposed_outputs = model.apply(params, inputs)
   one_hot = jax.nn.one_hot(outputs, VOCAB_DIM)
   loss = optax.softmax_cross_entropy(proposed_outputs, one_hot)
   return jnp.mean(loss)


def step(state, model, inputs, outputs):
   loss, grad = jax.value_and_grad(calculate_loss)(state.params, model, inputs, outputs)
   state = state.apply_gradients(grads = grad)
   return loss, state


mesh = jax.sharding.Mesh(np.reshape(  jax.devices(), (FSDP,TENSOR)), ["fsdp", "tp"])

ds = tfds.load('lm1b', split='train', shuffle_files=False)
ds = ds.batch(BATCH_IN_SEQUENCES)

rngkey = jax.random.key(0)
model = OurModel()

shaped_init = jax.eval_shape( functools.partial(model.init, rngkey), jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))
state_sharding = nn.get_sharding(shaped_init, mesh)
_params = jax.jit(model.init, out_shardings = state_sharding)(rngkey, jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))

tx = optax.adam(learning_rate = LEARNING_RATE)
state = train_state.TrainState.create(
    apply_fn = model.apply,
    params = _params,
    tx = tx
)

iter = 0
static_step = jax.jit(step, static_argnums=1)

last_step_time = time.time()
stepnum = 0

for example in ds:
    outputs = convert_to_ascii(example['text'].numpy(), SEQUENCE_LENGTH)
    inputs = input_to_output(outputs)

    loss, state = static_step(state, model, inputs, outputs)
    # assert model_without_pos_embeddings("Sam and Erin") == model_without_pos_embeddings("Erin and Sam")
    # assert model_with_pos_embeddings("Sam and Erin") != model_with_pos_embeddings("Erin and Sam")
    #loss, state = jax.jit(step, static_argnums=1)(state, model, inputs, outputs)
    #loss, state = jax.jit(lambda x,y,z,a : step(x,y,z,a), static_argnums=1)(state, model, inputs, outputs)

    stepnum += 1

    if stepnum % 10 == 0:
      new_time = time.time()
      time_elapsed_seconds = (new_time-last_step_time)
      last_step_time = new_time
      print(f"{iter} -> {loss} {time_elapsed_seconds}")


    iter += 1
    if stepnum == 1000:
      break


9 -> 4.641420364379883 10.499782800674438
19 -> 3.4807138442993164 0.4404611587524414
29 -> 3.126253604888916 0.4737560749053955
39 -> 2.970153570175171 0.4640030860900879
49 -> 2.615723133087158 0.4700591564178467
59 -> 2.5520169734954834 0.49142980575561523
69 -> 2.589102268218994 0.4575340747833252
79 -> 2.584998607635498 0.46743297576904297
89 -> 2.5628955364227295 0.4687764644622803
99 -> 2.529839277267456 0.46895742416381836
109 -> 2.508159637451172 0.4701712131500244
119 -> 2.8094897270202637 0.4691286087036133
129 -> 3.0647389888763428 0.4703054428100586
139 -> 2.7343554496765137 0.46842312812805176
149 -> 2.574105978012085 0.4739658832550049
159 -> 2.5387516021728516 0.46845340728759766
169 -> 2.547395706176758 0.46853137016296387
179 -> 2.5696353912353516 0.47060084342956543
189 -> 2.5902185440063477 0.4735286235809326
199 -> 2.4601614475250244 0.47171998023986816
209 -> 2.4510066509246826 0.4686911106109619
219 -> 2.786611318588257 0.4655430316925049
229 -> 2.488659381866455

In [32]:
import tensorflow as tf

def predict(input_str, model, params):
    input_tf = tf.constant([input_str], shape=(1,), dtype=tf.string)
    input_ascii = convert_to_ascii(input_tf.numpy(), SEQUENCE_LENGTH)
    logits = model.apply(params, input_ascii)
    return logits

# for example in ds:
#     print(example["text"][1:2])
#     inputs = convert_to_ascii(example['text'][1:2].numpy(), SEQUENCE_LENGTH)
#     break
logits_1 = predict("test a", model, state.params)
logits_2 = predict("test a", model, state.params)
logits_3 = predict("a test", model, state.params)
print(logits.shape)
print("logits 1 and 2 are the same:", jnp.array_equal(logits_1, logits_2))
print("logits 1 and 3 are NOT the same:", not jnp.array_equal(logits_1, logits_3))

(1, 128, 256)
logits 1 and 2 are the same: True
logits 1 and 3 are NOT the same: True


## Remove position embedding and rerun check

In [33]:
BATCH_IN_SEQUENCES = 384
SEQUENCE_LENGTH = 128

VOCAB_DIM = 256
EMBED_DIM = 512
FF_DIM = 2048

NUM_HEADS = 4
HEAD_DIM = 128

LAYERS = 2

HEAD_DEPTH = 128

LEARNING_RATE = 1e-3

FSDP = 4
TENSOR = 2


def attention_ourselves(_Q, _K, _V):
    _weights_unnormalized = jax.numpy.einsum("BSHD,BTHD->BHST", _Q, _K)
    _weights_unnormalized_to_zero_out = jax.numpy.triu( jax.numpy.ones((SEQUENCE_LENGTH,SEQUENCE_LENGTH), jax.numpy.bfloat16), 1)
    _weights = jax.nn.softmax(_weights_unnormalized - 1e6 * _weights_unnormalized_to_zero_out)  ### Creating something of size (B,HEADS, SEQUENCE, SEQUENCE)
    #print(f"{_weights.size=}")
    output = jax.numpy.einsum("BHST,BTHD->BSHD", _weights, _V)

    return output

class OurModel(nn.Module):
  @nn.compact
  def __call__(self, x):
    '''
        x is [BATCH, SEQUENCE]
    '''
    embedding = self.param(
        'embedding',
        nn.with_partitioning(nn.initializers.normal(1), ("tp", "fsdp")),
        (VOCAB_DIM, EMBED_DIM),
        jnp.float32,
    )
    x = embedding[x] ##OUTPUT should be [BATCH, SEQUENCE, EMBED]

    # positional_embedding = self.param(
    #     'positional_embedding',
    #     nn.with_partitioning(nn.initializers.normal(1), (None, None, "fsdp")),
    #     (1, SEQUENCE_LENGTH, EMBED_DIM),
    #     jnp.float32,
    # )

    # x += positional_embedding


    for i in range(LAYERS):
      feedforward = self.param(
          'feedforward_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, FF_DIM),
          jnp.float32,
      )
      x = x @ feedforward
      x = jax.nn.relu(x)
      embed = self.param(
          'embed_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('tp', 'fsdp')),
          (FF_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = x @ embed
      x = jax.nn.relu(x)

      q_proj = self.param(
          'qproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      q = jnp.einsum("BSE,EHD->BSHD",x, q_proj )

      k_proj = self.param(
          'kproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      k = jnp.einsum("BSE,EHD->BSHD",x, k_proj )

      v_proj = self.param(
          'vproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (EMBED_DIM, NUM_HEADS, HEAD_DIM),
          jnp.float32,
      )
      v = jnp.einsum("BSE,EHD->BSHD",x, v_proj )

      o = attention_ourselves(q,k,v)

      o_proj = self.param(
          'oproj_' + str(i),
          nn.with_partitioning(nn.initializers.lecun_normal(), ('fsdp', 'tp')),
          (NUM_HEADS, HEAD_DIM, EMBED_DIM),
          jnp.float32,
      )
      x = jnp.einsum("BSHD,HDE->BSE",o, o_proj )

    return x @ embedding.T # missing the softmax with probabilities for each token?

def convert_to_ascii(string_array, max_length):
  result = np.zeros((len(string_array), max_length), dtype=np.uint8)
  for i, string in enumerate(string_array):
    for j, char in enumerate(string):
      if j >= SEQUENCE_LENGTH:
         break
      result[i, j] = char
  return result

def input_to_output(np_array):
   zero_array = np.zeros( (BATCH_IN_SEQUENCES,SEQUENCE_LENGTH), dtype = jnp.uint8)
   zero_array[:, 1:SEQUENCE_LENGTH] = np_array[:, 0:SEQUENCE_LENGTH-1]
   return zero_array

def calculate_loss(params, model, inputs, outputs):
   proposed_outputs = model.apply(params, inputs)
   one_hot = jax.nn.one_hot(outputs, VOCAB_DIM)
   loss = optax.softmax_cross_entropy(proposed_outputs, one_hot)
   return jnp.mean(loss)


def step(state, model, inputs, outputs):
   loss, grad = jax.value_and_grad(calculate_loss)(state.params, model, inputs, outputs)
   state = state.apply_gradients(grads = grad)
   return loss, state


mesh = jax.sharding.Mesh(np.reshape(  jax.devices(), (FSDP,TENSOR)), ["fsdp", "tp"])

ds = tfds.load('lm1b', split='train', shuffle_files=False)
ds = ds.batch(BATCH_IN_SEQUENCES)

rngkey = jax.random.key(0)
model = OurModel()

shaped_init = jax.eval_shape( functools.partial(model.init, rngkey), jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))
state_sharding = nn.get_sharding(shaped_init, mesh)
_params = jax.jit(model.init, out_shardings = state_sharding)(rngkey, jax.ShapeDtypeStruct((BATCH_IN_SEQUENCES, SEQUENCE_LENGTH), dtype = jnp.uint8))

tx = optax.adam(learning_rate = LEARNING_RATE)
state = train_state.TrainState.create(
    apply_fn = model.apply,
    params = _params,
    tx = tx
)

iter = 0
static_step = jax.jit(step, static_argnums=1)

last_step_time = time.time()
stepnum = 0

for example in ds:
    outputs = convert_to_ascii(example['text'].numpy(), SEQUENCE_LENGTH)
    inputs = input_to_output(outputs)

    loss, state = static_step(state, model, inputs, outputs)
    # assert model_without_pos_embeddings("Sam and Erin") == model_without_pos_embeddings("Erin and Sam")
    # assert model_with_pos_embeddings("Sam and Erin") != model_with_pos_embeddings("Erin and Sam")
    #loss, state = jax.jit(step, static_argnums=1)(state, model, inputs, outputs)
    #loss, state = jax.jit(lambda x,y,z,a : step(x,y,z,a), static_argnums=1)(state, model, inputs, outputs)

    stepnum += 1

    if stepnum % 10 == 0:
      new_time = time.time()
      time_elapsed_seconds = (new_time-last_step_time)
      last_step_time = new_time
      print(f"{iter} -> {loss} {time_elapsed_seconds}")


    iter += 1
    if stepnum == 1000:
      break


9 -> 3.738297939300537 10.284725666046143
19 -> 3.2976555824279785 0.4366028308868408
29 -> 2.9179940223693848 0.46828627586364746
39 -> 2.6275124549865723 0.4704246520996094
49 -> 2.5893445014953613 0.4669492244720459
59 -> 2.526007890701294 0.4689810276031494
69 -> 2.565831422805786 0.46811962127685547
79 -> 2.552938461303711 0.4696502685546875
89 -> 2.4898769855499268 0.4689011573791504
99 -> 2.4497549533843994 0.4737987518310547
109 -> 2.3568036556243896 0.4676961898803711
119 -> 2.33327579498291 0.4705238342285156
129 -> 2.2718138694763184 0.4678027629852295
139 -> 2.289827823638916 0.46984219551086426
149 -> 2.197004795074463 0.467543363571167
159 -> 2.1709423065185547 0.4693896770477295
169 -> 2.17602276802063 0.4683966636657715
179 -> 2.1939661502838135 0.47025537490844727
189 -> 2.231370687484741 0.464611291885376
199 -> 2.1232893466949463 0.4722745418548584
209 -> 2.1250648498535156 0.4691329002380371
219 -> 2.1651811599731445 0.46866273880004883
229 -> 2.094897985458374 0.46

In [34]:
import tensorflow as tf

def predict(input_str, model, params):
    input_tf = tf.constant([input_str], shape=(1,), dtype=tf.string)
    input_ascii = convert_to_ascii(input_tf.numpy(), SEQUENCE_LENGTH)
    logits = model.apply(params, input_ascii)
    return logits

# for example in ds:
#     print(example["text"][1:2])
#     inputs = convert_to_ascii(example['text'][1:2].numpy(), SEQUENCE_LENGTH)
#     break
logits_1 = predict("test a", model, state.params)
logits_2 = predict("test a", model, state.params)
logits_3 = predict("a test", model, state.params)
print(logits.shape)
print("logits 1 and 2 are the same:", jnp.array_equal(logits_1, logits_2))
print("logits 1 and 3 are NOT the same:", not jnp.array_equal(logits_1, logits_3))

(1, 128, 256)
logits 1 and 2 are the same: True
logits 1 and 3 are NOT the same: True
